In [40]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import folium

In [11]:
key = open('../keys/google.txt').readline()

In [2]:
damage = pd.read_csv('../data/Mapping Home Loss - Sheet1.csv')
print(damage.shape)
damage.head()

(292, 7)


,Address,Name,Before photos,After photos,Story,Notes,Unnamed: 6
0,Hickory Tree Road,NaN,NaN,NaN,NaN,NaN,NaN
1,Riverview Parkway,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,There is a trailer park east of River Knoll th...,NaN
3,Old Farm School Road,NaN,NaN,NaN,NaN,NaN,NaN
4,382 Old Farm School Road,NaN,NaN,NaN,NaN,displaced total loss,NaN


In [3]:
damage.tail()

,Address,Name,Before photos,After photos,Story,Notes,Unnamed: 6
287,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,Landslides in Grovemont,NaN,NaN,NaN,NaN,Can Sheri help?,NaN
289,"156 Dryid Hill Road, Swannanoa",James and Judy Dockery,NaN,NaN,NaN,killed in landslide,NaN
290,"Brierley Hill, Black Mountain",Sam Saunders,NaN,NaN,NaN,Landslides,NaN
291,"Brierley Hill, Black Mountain",Bill Midkiff,NaN,NaN,NaN,NaN,NaN


In [37]:
addresses = damage.Address
addresses[49:60]

49               NaN
50     206 Old US 70
51     207 Old Us 70
52     212 Old US 70
53     216 Old US 70
54     218 Old US 70
55    220 Old US 70 
56     222 Old US 70
57     226 Old US 70
58     228 Old US 70
59     232 Old US 70
Name: Address, dtype: object

In [34]:
test = ['109 Willow Lake Dr, Asheville, NC, 28805',
'129 Willow Lake Drive, Asheville, NC, 28805',
'127 Willow Lake Drive, Asheville, NC, 28805',
'125 Willow Lake Drive, Asheville, NC, 28805',
'123 Willow Lake Drive, Asheville, NC, 28805',
'121 Willow Lake Drive, Asheville, NC, 28805',
'117 Willow Lake Drive, Asheville, NC, 28805',
'115 Willow Lake Drive, Asheville, NC, 28805',
'111 Willow Lake Drive, Asheville, NC, 28805',
'107 Willow Lake Drive, Asheville, NC, 28805',
'105 Willow Lake Drive, Asheville, NC, 28805']

In [38]:
def get_coordinates(key, address):
    endpoint_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    params = {
        'address': address,
        'key': key
    }
    try:
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        return results
    except Exception as e:
        print('error: ', address, e) 

In [43]:
geo_addr = []
for address in addresses:
    geo_addr.append(get_coordinates(key, address))

In [45]:
print(geo_addr[0]['results'][0]['geometry']['location'])

{'lat': 28.1971627, 'lng': -81.2005487}


In [46]:
geometries = []
error_list = []
for i in range(len(geo_addr)):
    try:
        geometries.append(geo_addr[i]['results'][0]['geometry']['location'])
    except:
        error_list.append(i)
        print('cannot add' , i)

cannot add 2
cannot add 33
cannot add 35
cannot add 37
cannot add 39
cannot add 40
cannot add 41
cannot add 42
cannot add 44
cannot add 45
cannot add 47
cannot add 48
cannot add 49
cannot add 66
cannot add 74
cannot add 78
cannot add 79
cannot add 84
cannot add 86
cannot add 89
cannot add 93
cannot add 99
cannot add 101
cannot add 102
cannot add 106
cannot add 112
cannot add 113
cannot add 117
cannot add 119
cannot add 120
cannot add 121
cannot add 136
cannot add 137
cannot add 140
cannot add 141
cannot add 142
cannot add 143
cannot add 144
cannot add 145
cannot add 147
cannot add 149
cannot add 151
cannot add 162
cannot add 163
cannot add 164
cannot add 165
cannot add 166
cannot add 167
cannot add 168
cannot add 169
cannot add 170
cannot add 176
cannot add 177
cannot add 179
cannot add 181
cannot add 187
cannot add 198
cannot add 199
cannot add 200
cannot add 203
cannot add 207
cannot add 208
cannot add 210
cannot add 212
cannot add 213
cannot add 219
cannot add 221
cannot add 223
can

In [47]:
geo_addr[33]

{'results': [], 'status': 'ZERO_RESULTS'}

In [48]:
len(geo_addr)

292

In [49]:
len(error_list)

94

In [50]:
damage = damage.drop(error_list)

In [51]:
damage['geometry'] = [Point(geo['lng'], geo['lat']) for geo in geometries]

In [52]:
damage.head()

,Address,Name,Before photos,After photos,Story,Notes,Unnamed: 6,geometry
0,Hickory Tree Road,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-81.2005487 28.1971627)
1,Riverview Parkway,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-116.9786411 32.8411904)
3,Old Farm School Road,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-82.4682957 35.6071633)
4,382 Old Farm School Road,NaN,NaN,NaN,NaN,displaced total loss,NaN,POINT (-82.4701133 35.6079282)
5,392 Old Farm School Road,Amanda Ogle,NaN,NaN,NaN,displaced total loss,NaN,POINT (-82.46942299999999 35.6078078)


In [64]:
damage['lng'] = damage.geometry.apply(lambda p: p.x)
damage['lat'] = damage.geometry.apply(lambda p: p.y)

In [65]:
center = [35.6011002,-82.4151444]
helene = folium.Map(location = center, zoom_start = 12)
helene

In [69]:
for row_index, row_values in damage.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['Address'])
    if row_values['Notes'] == 'displaced total loss':
        color = 'red'
    else:
        color = 'orange'
    icon=folium.Icon(color=color,icon=" fa-exclamation-circle", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(helene)
helene.save('../maps/helene_map.html')

helene